In [ ]:
// #r "C:\experimental\internal\src\private-seb\Notebooks\XESF\HPC\binaries\BoSSSpad.dll"
// #r "C:\experimental\internal\src\private-seb\Notebooks\XESF\HPC\binaries\XESF.dll"
// #r "C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\internal\src\private-seb\SAIDT\bin\Release\net6.0\BoSSSpad.dll"
// #r "C:\Users\jakob\Documents\Uni\Promotion\Programmieren\BoSSS\experimental\internal\src\private-seb\SAIDT\bin\Release\net6.0\SAIDT.dll"
// #r "..\..\..\..\..\..\..\..\internal\src\private-seb\SAIDT\bin\Release\net6.0\BoSSSpad.dll"
// #r "..\..\..\..\..\..\..\..\internal\src\private-seb\SAIDT\bin\Release\net6.0\SAIDT.dll"
#r "BoSSSpad.dll"
#r "SAIDT.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using SAIDT;
Init();


In [ ]:
string PROJECT_NAME = System.Environment.GetEnvironmentVariable("ShockFitting") ?? "ShockFitting"; // this allows to modify the project name for testing purposes
wmg.Init(PROJECT_NAME);
wmg.SetNameBasedSessionJobControlCorrelation();
wmg.AllJobs

## Utility definitions

In [ ]:
using BoSSS.Foundation.Grid;
static class Utils {
    // DOF per cell for one variable
    public static int Np(int p) {
        return (p*p + 3*p + 2)/2;
    }    
    public static int Nq_SpecFemField(int q){
        return (q*q + 3*q + 2)/2;
    }
    public static int Nq_SplineLevelSet(int YNodes, int q){
        int numberOfNodes = YNodes;

        //dependent on the degree we have more free Parameters
        if(1 == 0) {
            throw new NotSupportedException("degree 0 not allowed");
        } else if(q == 1) {
            numberOfNodes =YNodes;
        } else if(q == 2) {
            numberOfNodes =YNodes * 2;
        } else {
            numberOfNodes =YNodes * 2;
        }
        return numberOfNodes;
    }

}

## Init grids and save to database

In [ ]:
int[] Resolutions_2D = new int[] { 4,8 ,16 };//, 24};


In [ ]:
wmg.DefaultDatabase

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
int[] PolyDegS = new int[] {0, 1, 2};
int[] LevelSetDegs = new int[] {1,2, 3};

LevelSetTypes to test

In [ ]:
using ApplicationWithIDT;
OptiLevelSetType[] LStypes = new OptiLevelSetType[] {
    //OptiLevelSetType.GlobalLevelSet,
    OptiLevelSetType.SpecFemField,
    OptiLevelSetType.SplineLevelSet};
    LStypes

Merit Functions to Test

In [ ]:
MeritFunctionType[] MeritFunctionTypes = new MeritFunctionType[] {
    MeritFunctionType.ExactMerit, 
    MeritFunctionType.FullyL2Merit};
    MeritFunctionTypes

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int GetMaxAllowedDOF() {
    return 50000;
}

Rigth now the creation of control files does not work

In [ ]:
var myBatch = GetDefaultQueue();
myBatch

In [ ]:
int noOfTS =50;
bool[] Configs = new bool[] {true,false};


foreach(int res in Resolutions_2D) {
foreach(int p in PolyDegS) {
foreach(int q in LevelSetDegs) {
foreach(OptiLevelSetType lstype in LStypes) {
foreach(MeritFunctionType mfType in MeritFunctionTypes) {
foreach(bool config in Configs) {
        int Np = Utils.Np(p);
        int Nq=1;
        int J    = res*res;
        int numOfXCells = res;
        switch(lstype){
            case OptiLevelSetType.SplineLevelSet:
            Nq = Utils.Nq_SplineLevelSet(numOfXCells,q);
            if(q==2){
                continue;
            }
            break;
            case OptiLevelSetType.SpecFemField:
            Nq = q*J;
            if(q==1){
                continue;
            }
            break;
            case OptiLevelSetType.SinglePhaseField:
            Nq = Utils.Np(q)*J;
            if(q==2){
                continue;
            }
            break;
            // not cell dependant
            case OptiLevelSetType.GlobalLevelSet:
            Nq=Utils.Np(q);
            break;
        }
        int DOF  = Np*2 + Nq;
        
        if(DOF >  GetMaxAllowedDOF())
            continue;
        // Control Instance, grid, DG degree, etc.
        // =======================================

    Job j = new Job(string.Format("SAIDT-J{0}_p{1}_q{2}_{3}_{4}_isFarConfig{5}", J, p, q,lstype,mfType,config), typeof(SAIDT.SAIDTMain));// job name has to be equal to sessionname! for SessionInfoJobCorrelation to work!
    j.RetryCount = 1;
    string command = $"cs:SAIDT.SAIDTHardCodedControl.StraightShock("+
    $"dbPath:{"@\""+BoSSSshell.WorkflowMgm.DefaultDatabase.Path+"\""},"+
    $"NoOfTimesteps:{noOfTS},"+
    $"dgDegree:{p},"+
    $"numOfCellsX:{numOfXCells},numOfCellsY:{numOfXCells},"+
    $"OptiNumOfCellsX : {numOfXCells},OptiNumOfCellsY :{numOfXCells},"+
    $"PrintInterval:{-1},"+
    $"optiLevelSetType:ApplicationWithIDT.OptiLevelSetType.{lstype},"+
    $"LSDegree : {q},"+
    $"withReInit: {"true"},"+ 
    $"agg : {0.1},"+
    $"meritFunctionType: ApplicationWithIDT.MeritFunctionType.{mfType},"+
    $"isFarConfig:{config.ToString().ToLower()}" +
    ")";
    Console.WriteLine("$\"" + command +"\"");

    j.MySetCommandLineArguments("--control",  command); // db path has to be valid at remote location as well! otherwise fix this somehow to pathatremote

    j.Activate(myBatch); 
}
}
}
}
}
}

In [ ]:
wmg.AllJobs.Count

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(TimeOutSeconds:(3600*24*5), PollingIntervallSeconds:(60*5));


In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => Si.SuccessfulTermination == false
    || Convert.ToInt32(Si.KeysAndQueries["Conv"]) == 0);
FailedSessions

In [ ]:
//// The Session column can't be serialized,
//// we have to remove it
//List<string> AllCols = FullSessTab.GetColumnNames().ToList();
//AllCols.Remove("Session");
//FullSessTab = FullSessTab.ExtractColumns(AllCols.ToArray());


List of all available Data Columns:

In [ ]:
// var SessTab = wmg.DefaultDatabase.Sessions.GetSessionTable();
// SessTab.GetColumnNames().ToConcatString("", "; ", "")

Select those columns which are of interest:

In [ ]:

// var SubTab = SessTab.ExtractColumns(    "SessionName", "DGdegree:levelSet", "DgDegree", "OptiLevelSetDegree","Grid:NoOfCells", 
//     "Gamma_Start", "Alpha_Start", "Mu_Start", "Gamma_Min", "reInit_c1", "reInit_c2", "reInit_c3", "L", 
//     "GetInitialValue","GetLevelSet","OptiLevelSetType","GlobalizationStrategy","FDtype","MeritFunctionType",
//     "sigma_1","sigma_2","tauGamma","Alpha_Min","Gamma_Max","Linearization","NoOfTimesteps");
// SubTab.GetColumnNames().ToConcatString("", "; ", "")

## Create Plots

In [ ]:
var dTypes =new List<DashTypes>();
dTypes.Add(DashTypes.Dashed);dTypes.Add(DashTypes.Solid);dTypes.Add(DashTypes.DotDotDashed);dTypes.Add(DashTypes.Dotted);
dTypes.Add(DashTypes.Solid);dTypes.Add(DashTypes.Dashed);dTypes.Add(DashTypes.DotDotDashed);dTypes.Add(DashTypes.Dotted);

var pTypes =new List<PointTypes>();
pTypes.Add(PointTypes.Asterisk);pTypes.Add(PointTypes.Diamond);pTypes.Add(PointTypes.Box);pTypes.Add(PointTypes.Circle);
pTypes.Add(PointTypes.Plus);pTypes.Add(PointTypes.OpenCircle);pTypes.Add(PointTypes.UpperTriangle);pTypes.Add(PointTypes.LowerTriangle);

var pColors =new List<LineColors>();
pColors.Add(LineColors.Black);pColors.Add(LineColors.Red);pColors.Add(LineColors.Blue);pColors.Add(LineColors.Green);
pColors.Add(LineColors.Magenta);pColors.Add(LineColors.Black);pColors.Add(LineColors.Red);pColors.Add(LineColors.Red);


Func<double[],double> StraightShockExactSol = delegate(double[] x){
    if(x[0] <0.25 + 0.5*x[1]){
        return 1;
    }else{
        return 0;
    }
};
Func<ITimestepInfo,double> StraightShockErrorFunctional = delegate(ITimestepInfo ts){
    return ts.GetField("c").L2Error(StraightShockExactSol);
};
var plots = new List<Plot2Ddata>();
Plot2Ddata[,] multiplots =new Plot2Ddata[LStypes.Count(),PolyDegS.Length];

foreach(int p in PolyDegS) {
int countPoly =0;
foreach(MeritFunctionType mfType in MeritFunctionTypes) {
Plot2Ddata plot = new Plot2Ddata();
plot.Title=mfType.ToString();
plot = plot.WithLogY();
int countMerit =0;
int count=0;
foreach(OptiLevelSetType lstype in LStypes) {
foreach(int q in LevelSetDegs) {
foreach(bool config in Configs) {
    var sess =wmg.DefaultDatabase.Sessions.Where(Si => Convert.ToBoolean(Si.KeysAndQueries["isFarConfig"]) ==config && 
    Convert.ToInt32(Si.KeysAndQueries["DgDegree"])==p &&
    Convert.ToInt32(Si.KeysAndQueries["OptiLevelSetType"])==lstype.GetHashCode() &&
    Convert.ToInt32(Si.KeysAndQueries["MeritFunctionType"])==mfType.GetHashCode() &&
    Convert.ToInt32(Si.KeysAndQueries["OptiLevelSetDegree"])==q);
    var xval= new List<double>();
    var yval= new List<double>();
    if(sess.Count()>0){
        foreach(SessionProxy ses in sess){
            var ti = ses.Timesteps.Last();
            xval.Add(Convert.ToInt32(ses.KeysAndQueries["Grid:NoOfCells"]));
            try{
                yval.Add(StraightShockErrorFunctional( ti));
            }catch{
                yval.Add(1e6);
            }
        }
        plot.AddDataGroup(xval,yval);
        if(config){
            plot.dataGroups[count].Name =  lstype.ToString() + "-p"+p+ "-q"+q +"-Far";
        }
        else{
            plot.dataGroups[count].Name =  lstype.ToString() + "-p"+p+ "-q"+q +"-Close";
        }
        plot.dataGroups[count].Format.PointType = pTypes[count]; 
        plot.dataGroups[count].Format.LineColor = pColors[count]; 
        plot.dataGroups[count].Format.PointSize = 0.8;
        plot.dataGroups[count].Format.LineWidth = 2;
        plot.dataGroups[count].Format.DashType = dTypes[count] ;
        count++;
        
        
    }
    
    //Console.WriteLine(sess.Count() + "; "+xval.ToConcatString("",",",""));
    
}}}
plot.LegendFont = 10;
plot.LabelFont = 10;
multiplots[countPoly,countMerit] = plot;
}
}

In [ ]:
multiplots.ToGnuplot().PlotSVG(xRes:1000,yRes:1600)

In [ ]:
// Plot2Ddata[,] multiplots2 =new Plot2Ddata[1,2];
// multiplots2[0,0]=multiplots[0,0];
// multiplots2[0,0].LegendFont = 10;
// multiplots2[0,0].LabelFont = 10;
// multiplots2[0,1]=multiplots[0,0];
// multiplots2.ToGnuplot().PlotSVG(xRes:1000,yRes:400)